In [1]:
import pyspark

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [5]:
days_list = []
for i in range(1,31):
    day_num = str(i) if len(str(i))==2 else '0'+str(i)
    day = sc.textFile('file:///data/mobile/raw_data/day%s.txt' % day_num).map(lambda x:x.split(','))
    days_list.append(day)
days_list[1].take(5)

[['BB1657CFD3B4B7208674BEB7D26014B2',
  '2820',
  'usr',
  '2',
  '07:03:37',
  '2',
  '07:05:23',
  '106',
  '316668',
  '47456'],
 ['BB1657CFD3B4B7208674BEB7D26014B2',
  '20858',
  'usr',
  '2',
  '07:05:38',
  '2',
  '07:11:27',
  '348',
  '1915724',
  '105292'],
 ['BB1657CFD3B4B7208674BEB7D26014B2',
  '15841',
  'usr',
  '2',
  '07:12:18',
  '2',
  '07:12:20',
  '2',
  '0',
  '0'],
 ['BB1657CFD3B4B7208674BEB7D26014B2',
  '15841',
  'usr',
  '2',
  '07:12:22',
  '2',
  '07:12:22',
  '3',
  '0',
  '0'],
 ['BB1657CFD3B4B7208674BEB7D26014B2',
  '20858',
  'usr',
  '2',
  '07:12:23',
  '2',
  '07:14:11',
  '108',
  '61059',
  '11193']]

In [8]:
days_ = sc.emptyRDD()
for i in range(23,30):
    days_ = days_.union(days_list[i].map(lambda x:(x[0]+'-'+x[1],1)))
y = days_.reduceByKey(lambda x,y:1)
y.take(5)

[('BF279627F55747C654F5189DF3A7A25A-6588', 1),
 ('A01CDC9E3665AB656124CA42286EF364-11088', 1),
 ('95B156DFF328E67C5AEE11221C6ED56A-16639', 1),
 ('EDA0269D6ABF7CE6CFAE029CFFB61423-3309', 1),
 ('FB7B9BA47A5C652338C50409E67EA4C7-20720', 1)]

In [34]:
_days = sc.emptyRDD()
for i in range(23):
    _days = _days.union(days_list[i].map(lambda x:(x[0]+'-'+x[1],(23-i,int(x[-3])))).reduceByKey(lambda x,y:(x[0],x[1]+y[1])))
_days.take(5)

[('BB1657CFD3B4B7208674BEB7D26014B2-18386', (23, 524)),
 ('BB1657CFD3B4B7208674BEB7D26014B2-16759', (23, 14083)),
 ('BB1657CFD3B4B7208674BEB7D26014B2-6597', (23, 101)),
 ('210BE886FAD8E61BC45475DDC7B62670-3309', (23, 14975)),
 ('214678BE5961F27D2C57F1987C60BAE2-8832', (23, 14))]

In [35]:
x1_2 = _days.reduceByKey(lambda x,y:(y[0],y[1]) if y[0]<x[0] else (x[0],x[1]))
x1_2.take(5)

[('98E26BB54ABC5AEDCDFF20B1971C88FB-18024', (1, 1553)),
 ('A50F35DDB18ED612C71D692E5FE7524E-11088', (1, 28)),
 ('0CDC7984639DC3BF0B82E96F81A290E8-11088', (1, 314)),
 ('73AA0F1EF183333CF6DE14FD9912175B-10686', (1, 607)),
 ('CA77140BE312992B5547625A22719FE7-5205', (1, 42))]

In [37]:
_days.countByKey?

In [54]:
x3 = _days.map(lambda x:(x[0],1)).reduceByKey(lambda x,y:x+y).map(lambda x:(x[0],x[1]/23))
x3.take(5)

[('98E26BB54ABC5AEDCDFF20B1971C88FB-18024', 0.8695652173913043),
 ('A50F35DDB18ED612C71D692E5FE7524E-11088', 0.6956521739130435),
 ('0CDC7984639DC3BF0B82E96F81A290E8-11088', 0.9130434782608695),
 ('73AA0F1EF183333CF6DE14FD9912175B-10686', 0.8260869565217391),
 ('CA77140BE312992B5547625A22719FE7-5205', 0.6521739130434783)]

In [43]:
x4 = _days.reduceByKey(lambda x,y:(0,x[1]+y[1]))\
.map(lambda x:(x[0],x[1][1]/23))
x4.take(5)

[('98E26BB54ABC5AEDCDFF20B1971C88FB-18024', 1543.0434782608695),
 ('A50F35DDB18ED612C71D692E5FE7524E-11088', 66.3913043478261),
 ('0CDC7984639DC3BF0B82E96F81A290E8-11088', 194.47826086956522),
 ('73AA0F1EF183333CF6DE14FD9912175B-10686', 358.4782608695652),
 ('CA77140BE312992B5547625A22719FE7-5205', 47.82608695652174)]

In [44]:
x5 = _days.map(lambda x:(x[0],(1,x[1][1]))).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))\
.map(lambda x:(x[0],x[1][1]/x[1][0]))
x5.take(5)

[('98E26BB54ABC5AEDCDFF20B1971C88FB-18024', 1774.5),
 ('A50F35DDB18ED612C71D692E5FE7524E-11088', 95.4375),
 ('0CDC7984639DC3BF0B82E96F81A290E8-11088', 213.0),
 ('73AA0F1EF183333CF6DE14FD9912175B-10686', 433.94736842105266),
 ('CA77140BE312992B5547625A22719FE7-5205', 73.33333333333333)]

In [45]:
x6 = _days.filter(lambda x:x[1][0]==1).map(lambda x:(x[0],x[1][1]))
x6.take(5)

[('C504A4C2350DCCC09D89077D68BA1169-20761', 614),
 ('C0D7EC409096A42DC3754D2EEC372A46-13043', 137),
 ('C0D7EC409096A42DC3754D2EEC372A46-20720', 29),
 ('9AEF49D425761C04EC656E26BFF64153-3309', 1617),
 ('B0AB7168E0057CE0D64C180A3CE483FB-20761', 18626)]

In [58]:
x7 = _days.filter(lambda x:x[1][0]<=7).map(lambda x:(x[0],(1,x[1][1]))).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))\
.map(lambda x:(x[0],x[1][1]/x[1][0]))
x7.take(5)

[('5438B4D7AD30A6E66E7C19575E1B88C5-21820', 15528.666666666666),
 ('3AFCA61EC923AF6FE390B247420269B3-5959', 1148.8333333333333),
 ('61305E6A64E6DE1CB3A3D78FAA714FBC-9098', 812.0),
 ('A81CB2210A4DE66EDB450E4AFECB9F0D-10432', 268.5),
 ('D9FAAEF69375838B39C5C23E5F917620-8165', 2008.6)]

In [59]:
x8 = _days.filter(lambda x:x[1][0]>7).map(lambda x:(x[0],(1,x[1][1]))).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))\
.map(lambda x:(x[0],x[1][1]/x[1][0]))
x8.take(5)

[('98E26BB54ABC5AEDCDFF20B1971C88FB-18024', 1860.1333333333334),
 ('A50F35DDB18ED612C71D692E5FE7524E-11088', 75.3),
 ('0CDC7984639DC3BF0B82E96F81A290E8-11088', 238.14285714285714),
 ('73AA0F1EF183333CF6DE14FD9912175B-10686', 401.84615384615387),
 ('CA77140BE312992B5547625A22719FE7-5205', 79.41666666666667)]

In [49]:
rdd1 = sc.parallelize([("1", ("Spark", "beijing")), ("2", ("Hadoop", "shanghai")),("4", ("Hadoop", "shanghai"))])
rdd2 = sc.parallelize([("1", "Spark"), ("2",  "shanghai"),("5", ("Hadoop", "shanghai"))])
rdd1.join(rdd2).collect()

[('1', (('Spark', 'beijing'), 'Spark')),
 ('2', (('Hadoop', 'shanghai'), 'shanghai'))]

In [50]:
rdd1.join?

In [52]:
rdd1.keys().take(5)

['1', '2', '4']

In [ ]:
rdd1.

In [56]:
table_keys = x4.keys().collect()
table_keys[:5]

['98E26BB54ABC5AEDCDFF20B1971C88FB-18024',
 'A50F35DDB18ED612C71D692E5FE7524E-11088',
 '0CDC7984639DC3BF0B82E96F81A290E8-11088',
 '73AA0F1EF183333CF6DE14FD9912175B-10686',
 'CA77140BE312992B5547625A22719FE7-5205']

In [64]:
rdd_dist = {'y':y,'x1_2':x1_2,'x3':x3,'x5':x5,'x6':x6,'x7':x7,'x8':x8}
miss_list_dist = {'y':[],'x1_2':[],'x3':[],'x5':[],'x6':[],'x7':[],'x8':[]}
null_dist = {'y':0,'x1_2':(24,0),'x3':0,'x5':0,'x6':0,'x7':0,'x8':0}
new_dict = {}
table_keys = set(table_keys)
for str_,rdd in rdd_dist.items():
    print(str_)
    tmp_keys = set(rdd.keys().collect())
    print('collect done')
    print(len(tmp_keys))
#     for key in table_keys:
#         if key not in tmp_keys:
#             miss_list_dist[str_].append(key)
    miss_list_dist[str_] = table_keys - tmp_keys
    print('for done')
    tmp_rdd = sc.parallelize([(key,null_dist[str_]) for key in miss_list_dist[str_]])
    print('parallelize done')
    new_rdd = rdd.union(tmp_rdd)
    print('union done')
    new_dict[str_] = new_rdd
    print('done')
new_dict['x4'] = x4

y
collect done
1118655
for done
parallelize done
union done
done
x1_2
collect done
1741472
for done
parallelize done
union done
done
x3
collect done
1741472
for done
parallelize done
union done
done
x5
collect done
1741472
for done
parallelize done
union done
done
x6
collect done
478314
for done
parallelize done
union done
done
x7
collect done
1083785
for done
parallelize done
union done
done
x8
collect done
1541854
for done
parallelize done
union done
done


In [65]:
final_rdd = sc.emptyRDD().union(new_dict['y'])
for str_,rdd in new_dict.items():
    if str_ != 'y':
        final_rdd = final_rdd.join(rdd)
final_rdd.take(5)

[('044E7597FF48E6B7502A702E3DABDB21-4983',
  (((((((0, (8, 83)), 0.08695652173913043), 213.0), 0), 0), 213.0),
   18.52173913043478)),
 ('B9618CA6127006B80ACC088BA5A91C68-13983',
  (((((((0, (2, 77)), 0.043478260869565216), 77.0), 0), 77.0), 0),
   3.347826086956522)),
 ('5A9034ADADAF1A0F70D4DDDDD440E9D0-21004',
  (((((((0, (18, 825)), 0.08695652173913043), 841.0), 0), 0), 841.0),
   73.1304347826087)),
 ('66DEB41FFE7BE7A3643E8C3208FAB759-11088',
  (((((((1, (7, 42)), 0.043478260869565216), 42.0), 0), 42.0), 0),
   1.826086956521739)),
 ('77E68627E40041D3B64A255D4D201EFE-3427',
  (((((((1, (6, 43)), 0.08695652173913043), 25.0), 0), 43.0), 7.0),
   2.1739130434782608))]

In [68]:
real_final = final_rdd.map(lambda x:(x[0],x[1][0][0][0][0][0][0][0],x[1][0][0][0][0][0][0][1][0]\
                                    ,x[1][0][0][0][0][0][0][1][1],x[1][0][0][0][0][0][1],x[1][0][0][0][0][1],x[1][0][0][0][1]\
                                    ,x[1][0][0][1],x[1][0][1],x[1][1]))
real_final.take(5)

[('044E7597FF48E6B7502A702E3DABDB21-4983',
  0,
  8,
  83,
  0.08695652173913043,
  213.0,
  0,
  0,
  213.0,
  18.52173913043478),
 ('B9618CA6127006B80ACC088BA5A91C68-13983',
  0,
  2,
  77,
  0.043478260869565216,
  77.0,
  0,
  77.0,
  0,
  3.347826086956522),
 ('5A9034ADADAF1A0F70D4DDDDD440E9D0-21004',
  0,
  18,
  825,
  0.08695652173913043,
  841.0,
  0,
  0,
  841.0,
  73.1304347826087),
 ('66DEB41FFE7BE7A3643E8C3208FAB759-11088',
  1,
  7,
  42,
  0.043478260869565216,
  42.0,
  0,
  42.0,
  0,
  1.826086956521739),
 ('77E68627E40041D3B64A255D4D201EFE-3427',
  1,
  6,
  43,
  0.08695652173913043,
  25.0,
  0,
  43.0,
  7.0,
  2.1739130434782608)]

In [71]:
with open('/data/mobile/processed_data/yx1_8.txt','w') as f:
    for tu in real_final.collect():
        f.write(','.join([str(item) for item in tu])+'\n')

In [72]:
from pyspark.mllib.regression import LabeledPoint

In [76]:
for_training = real_final.map(lambda x: LabeledPoint(float(x[1]),[float(x[i]) for i in range(2,10)]))
for_training.take(5)

[LabeledPoint(0.0, [8.0,83.0,0.08695652173913043,213.0,0.0,0.0,213.0,18.52173913043478]),
 LabeledPoint(0.0, [2.0,77.0,0.043478260869565216,77.0,0.0,77.0,0.0,3.347826086956522]),
 LabeledPoint(0.0, [18.0,825.0,0.08695652173913043,841.0,0.0,0.0,841.0,73.1304347826087]),
 LabeledPoint(1.0, [7.0,42.0,0.043478260869565216,42.0,0.0,42.0,0.0,1.826086956521739]),
 LabeledPoint(1.0, [6.0,43.0,0.08695652173913043,25.0,0.0,43.0,7.0,2.1739130434782608])]

In [102]:
(trainData, testData) = for_training.randomSplit([8,2])
print("训练集样本个数："+str(trainData.count()) + "测试集样本个数："+str(testData.count()))

# 将数据暂存在内存中，加快后续运算效率
trainData.persist()
testData.persist()

训练集样本个数：1393518测试集样本个数：347954


PythonRDD[1660] at RDD at PythonRDD.scala:53

In [90]:
from pyspark.mllib.tree import RandomForest

In [121]:
rf = RandomForest.trainClassifier(sc.parallelize(trainData), 2, {}, 3, seed=42)

KeyboardInterrupt: 

In [104]:
print(f'intercept: {lrm.intercept}')
print(f'coeffs: {lrm.weights}')

intercept: 0.0
coeffs: [-6.599803309256028,-850.9659640096515,0.21681646603047455,-7.353918733245337,7531.449445688068,2630.158661119206,4009.327432071751,6207.529855853209]


In [113]:
test_data = testData.take(5)
test_data

[LabeledPoint(0.0, [2.0,77.0,0.043478260869565216,77.0,0.0,77.0,0.0,3.347826086956522]),
 LabeledPoint(1.0, [1.0,441.0,0.782608695652174,372.5,441.0,230.0,413.2142857142857,291.5217391304348]),
 LabeledPoint(0.0, [5.0,1138.0,0.08695652173913043,1952.0,0.0,1138.0,2766.0,169.7391304347826]),
 LabeledPoint(0.0, [1.0,39.0,0.043478260869565216,39.0,39.0,39.0,0.0,1.6956521739130435]),
 LabeledPoint(1.0, [2.0,8.0,0.391304347826087,61.888888888888886,0.0,15.666666666666666,85.0,24.217391304347824])]

In [108]:
test_data.features

DenseVector([2.0, 77.0, 0.0435, 77.0, 0.0, 77.0, 0.0, 3.3478])

In [120]:
lrm.clearThreshold()
lrm.predict(test_data[0].features)

1.0

In [ ]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics
score_label_lr = testData.map

In [95]:
from pyspark.mllib.classification import SVMWithSGD

In [126]:
svm = SVMWithSGD.train(trainData)

109075.50646850988

In [129]:
svm.se

In [132]:
from pyspark.mllib.tree import RandomForest

In [ ]:
rf = SVMWithSGD.train(trainData)